# **The Hough Transform [Part 2]**

**Recommended Reading/Viewing:**

1. [How the Hough Transform was Invented](http://www.rci.rutgers.edu/~shunsun/resource/Hough_Transform.pdf)

**Additional Reading**
1. Placeholder

**Recommended [Jupyter Theme](https://github.com/dunovank/jupyter-themes) for presenting this notebook:**
````
jt -t grade3 -cellw=90% -fs=20 -tfs=20 -ofs=20
````

---

Last time we left off with two questions:

![](../videos/hough_two.gif)

## 1. How can we use the Hough Transform to **efficiently** find **noisy** colinear points?
## 2. How might we modify the Hough Transform to avoid having intersection points at $\infty$?

![](../graphics/spacer_small.png)

- To anwers these questions, we need to visit SRI (Stanford Research Institute) in the late 1960s. 
- This was an interesting time at SRI. Researchers were building the **world's first mobile robot,** [**Shakey**](https://www.youtube.com/watch?v=GmU7SimFkpU).

![](../videos/shakey_1.gif)

- One of the many problems SRI researchers faced in developing Shakey was creating a vision system that would allow Shakey to interpet the "Blocks World" he operated in.

![](../videos/shakey_2.gif)

- SRI Researchers **Peter E. Hart** and **Richard O. Duda** were looking for an efficient and reliable method for detecting lines in images. 
- Hart + Duda likely used **Robert's Cross** or the **Sobel-Feldman Operator** to find edge pixels in images. 
- They were then trying to use these images to "build a model of the local environment and to update the position of the robot from identificaion of known landmarks."

![](../graphics/hart_and_duda-01.png)

- In search for a solution, **Peter Hart** studied the first computer vision book (!), **Azriel Rosenfeld's Picture Processing by Computer.**
- In this book, Rosenfeld briefly mentions the then obscure Hough patent, describing the tranformation algabraically for the first time, and posed a simple and computationally efficient method of implementing Hough's method. 


[Placeholder - Azriel Rosenfeld, Book, and relevant excerpt]

- However, Rosenfeld does **not solve the intersection at $\infty$ problem.**
- Simultaneously, Hart was examining another appraoch that he thought may be helpful in pattern recognition - **integral geometry.** This approach didn't work, but he did pick up a potentially useful alternative parameterization of a line.
- Let's have a look at this representation.

![](../graphics/integral_geometry_line_a.png)

- Instead of parameterizing our line with a slope and y-intercept, here we're parameterizing our line using $\rho$ and $\theta$.
     - $\rho$ is the distance from the origin to the closest point on our line
     - $\theta$ is the angle between the positive side of our x-axis and a the chord connecting the origin to our line at a right angle
- Now that we've established this alternative line representation, we have an important little math puzzle to solve:

# **What is the equation of our line in terms of $\rho$ and $\theta$ ?**

---

- Let's take this question one step at time. 
- First, we need a strategy. One strategy is to use our familiar parameterization of a line $y = mx +b$, and try to figure out our slope $m$ and our y-intercept $b$ in terms of our new parameters $\rho$ and $\theta$. 

![](../graphics/hough_question_three-01.png)

---

![](../graphics/hough_question_four-01.png)

---

![](../graphics/spacer_small.png)

![](../graphics/spacer_small.png)

Putting this all together, we have a new equation for a line in terms $\rho$ and $\theta$:

## $$
y = -\frac{cos(\theta)}{sin(\theta)} x + \frac{\rho}{sin(\theta)}
$$

We can re-arrange our new equation to make it a little more aesthetically pleasing:

## $$
\rho = y sin(\theta) + x cos(\theta)
$$

Let's make sure the form of our line makes sense:

![](../graphics/hough_question_five-01.png)

___

Now, here's where it gets interesting. Hough used the common $y=mx+b$ representation of the line as the core of his transform, leading to an unbounded transform space. What Peter Hart saw here was a way to use this alternate representation of a line, $\rho = y sin(\theta) + x cos(\theta)$, to tranform our data into a more useful version of Hough's original space. Let's have a look at this transform!

In [1]:
from IPython.display import Image, display
from ipywidgets import interact

def slide_show(slide_num=0):    
    '''Make a little slide show in the notebook'''
    display(Image('../graphics/hart_space/hart_space_' + str(slide_num) + '-01.png'))

In [2]:
interact(slide_show, slide_num = (0, 5));

A Jupyter Widget

- Using this alternate from of the line, our points are now transformed into pieces of sinusoids. 
- As you may know, the sum of two sinusiods of the same variable is another sinusoid of the same variable. 
    - It's useful here to know a little trigonometry or be comfortable with Euler's formula. 
- The really cool thing here is that, just as co-linear points mapped to intersecting lines with Hough's tranform, **co-linear points points map to intersecting curves here**.
- Now, this was true in Hough's original appraoch, remember that the real problem was that Hough's original space was unbounded - some co-linear points leds to lines that intersected at $\infty$.
- Let's see how our new space, introduced by Hart, handles this problem:

![](../videos/hough_three.gif)

- As you can see, in our $\rho \theta$ space, we see intersections for **both** horizontal and vertical lines.
- In fact, we can represent $any$ line in $x, y$ space as a point in bounded $\rho, \theta$ space!
- So we've turned our problem of finding interecting points into a problem of finding interecting **sinusoids.** and made some real progress!

![](../graphics/spacer_small.png)

## 1. How can we use the Hough Transform to **efficiently** find **noisy** colinear points?
## <s> 2. How might we modify the Hough Transform to avoid having intersection points at infinity? <s>

![](../graphics/spacer_small.png)

- Ok, one down one to go!
- Remember the whole idea here is to efficiently find noisy colinear points. This is what Hough set out to do for bubble chamber data, and what Hart and Duda needed to do to help shakey see make some sense of the world. 
- The answer to this problem comes not from Duda, Hart, or Hough, but from Azriel Rosenfeld, the author of the first computer vision book - Picture Processing by Computer. 

[Placeholder - Azriel Rosenfeld, Book, and relevant excerpt]

In [6]:
def slide_show(slide_num=0):    
    '''Make a little slide show in the notebook'''
    display(Image('../graphics/hough_accumulator/hough_accumulator_' + str(slide_num) + '.png'))

In [7]:
interact(slide_show, slide_num = (1, 3));

A Jupyter Widget

- Rosenfeld's presented a smart solution here - he first quantized the $\rho$, $\theta$ space into a grid
- Each cell in the grid represents an "accumulator".
- For each point in our image space $xy$, we map that point to a curver in $\rho \theta$ space, and increment the accumulator of each cell we pass through.
- Rosenfeld's solution is quite powerful. By giving up some accuracy, we're able to quickly compute an answer, and our quantization actualy makes our line detector **less susceptable to noise!** 

From [Wikipedia Hough Transform Article](https://en.wikipedia.org/wiki/Hough_transform): 

> *The purpose of the technique is to find imperfect instances of objects within a certain class of shapes by a voting procedure. This voting procedure is carried out in a parameter space, from which object candidates are obtained as local maxima in a so-called accumulator space that is explicitly constructed by the algorithm for computing the Hough transform.*
